In [ ]:
#!git clone https://github.com/huggingface/transformers.git#


In [ ]:
#pip install transformers=="4.0.0"

In [1]:
from fastai.text.all import *
import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import pandas as pd
import glob

/home/karynaur/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [ ]:
ids = tokenizer.encode('This is an example of text, and')
t = torch.LongTensor(ids)[None]
preds = model.generate(t)
tokenizer.decode(preds[0].numpy())


In [2]:
with open('/kaggle/input/dataforlyrics/21pilots/train.csv', encoding="utf8", errors='ignore') as f:
    train=pd.Series(f,index=None)
with open('/kaggle/input/dataforlyrics/21pilots/valid.csv', encoding="utf8", errors='ignore') as f:
    valid=pd.Series(f,index=None)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/dataforlyrics/21pilots/train.csv'

In [ ]:
train.shape,valid.shape


In [ ]:
train=train.dropna()
valid=valid.dropna()
train.head(),valid.head()

In [ ]:
all_texts = np.concatenate([train.values, valid.values])

all_texts.shape

In [ ]:
def tokenize(text):
    toks = tokenizer.tokenize(text)
    return tensor(tokenizer.convert_tokens_to_ids(toks))

tokenized = [tokenize(t) for t in progress_bar(all_texts)]

class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        return x if isinstance(x, Tensor) else tokenize(x)
        
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [ ]:
bs,sl = 4,256
splits = [range_of(train), list(range(len(train), len(all_texts)))]
tls = TfmdLists(tokenized, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
dls = tls.dataloaders(bs=bs, seq_len=sl)

In [ ]:
dls.show_batch(max_n=2)

In [ ]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [ ]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

In [ ]:
learn.validate()

In [ ]:
learn.path=Path('/')
learn.lr_find()

In [ ]:
learn.fit_one_cycle(20, 1e-4)

In [ ]:
learn.recorder.plot_loss()

In [ ]:
prompt = "\n uh "
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
preds = learn.model.generate(inp,
                             do_sample=True, 
                             max_length=90,
                             min_length=5,
                             top_k=40,
                             num_return_sequences=1)
tokenizer.decode(preds[0].cpu().tolist())


In [ ]:
learn.export('/kaggle/working/21pilots.pkl')

In [ ]:
#device=torch.device('cpu:0')
#learn.model.to(device)

In [31]:
with open('data/genius_lyrics_v2.csv', errors='ignore') as f:
    data=pd.read_csv(f)
data.shape

(15450, 6)

In [32]:
data.head()

,Genre,Artist,Year,Album,Song,Lyrics
0,Pop,Justin Bieber,2015,Purpose,Love Yourself,"For all the times that you rained on my parade\nAnd all the clubs you get in using my name\nYou think you broke my heart, oh girl for goodness sake\nYou think I'm cryin' on my own, well I ain't\n\nAnd I didn't wanna write a song\n'Cause I didn't want anyone thinking I still care\nI don't but, you still hit my phone up\nAnd baby I'll be movin' on\nAnd I think you should be somethin'\nI don't wanna hold back\nMaybe you should know that\n\nMy mama don't like you and she likes everyone\nAnd I never like to admit that I was wrong\nAnd I've been so caught up in my job, didn't see what's going on..."
1,Pop,Justin Bieber,2015,Purpose,Sorry,"You gotta go and get angry at all of my honesty\nYou know I try but I don't do too well with apologies\nI hope I don't run out of time, could someone call a referee?\nCause I just need one more shot at forgiveness\nI know you know that I made those mistakes maybe once or twice\nBy once or twice I mean maybe a couple a hundred times\nSo let me, oh let me redeem, oh redeem, oh myself tonight\nCause I just need one more shot at second chances\n\nYeah, is it too late now to say sorry?\nCause I'm missing more than just your body\nIs it too late now to say sorry?\nYeah I know that I let you down..."
2,Pop,Justin Bieber,2012,Believe,As Long As You Love Me,"As long as you love me\nAs long as you love me\nAs long as you love me\n\nWe're under pressure\nSeven billion people in the world tryna fit in\nKeep it together\nSmile on your face, even though your heart is frowning\nBut hey now, you know girl\nWe both know it's a cruel world\nBut I will take my chances\n\nAs long as you love me\nWe could be starving, we could be homeless, we could be broke\nAs long as you love me\nI'll be your platinum, I'll be your silver, I'll be your gold\nAs long as you love me\nAs long as you love me, love me\n\nI'll be your soldier\nFighting every second of the day..."
3,Pop,Justin Bieber,2015,Purpose,What Do You Mean?,"What do you mean? Oh, oh\nWhen you nod your head yes, but you wanna say no\nWhat do you mean? Hey-ey\nWhen you don't want me to move, but you tell me to go\nWhat do you mean?\nOh, what do you mean?\nSaid you're running outta time, what do you mean?\nOh, oh, oh, what do you mean?\nBetter make up your mind, what do you mean?\n\nYou’re so indecisive, what I’m saying\nTryna catch the beat, make up your heart\nDon't know if you're happy or complaining\nDon't want for us to end, where do I start?\n\nFirst you wanna go to the left then you wanna turn right\nWanna argue all day, make love all nigh..."
4,Pop,Justin Bieber,2010,My World 2.0,Baby,"Oh, woah\nOh, woah\nOh, woah\n\nYou know you love me, I know you care\nJust shout whenever and I’ll be there\nYou want my love, you want my heart\nAnd we will never, ever, ever be apart\nAre we an item? Girl, quit playing\nWe’re just friends, what are you saying?\nSaid ""There’s another,"" and looked right in my eyes\nMy first love broke my heart for the first time, and I was like...\n\nBaby, baby, baby oh\nLike baby, baby, baby no\nLike baby, baby, baby no oh\nThought you'd always be mine, mine\nBaby, baby, baby oh\nLike baby, baby, baby no\nLike baby, baby, baby no oh\nThought you’d always..."


In [23]:
!pip3 install jovian -q --upgrade
import jovian

In [33]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "karynaur/lyrics" on https://jovian.ai/
[jovian] Uploading notebook..
[jovian] Capturing environment..


[jovian] Error: Failed to read Anaconda environment using command: "conda env export -n base --no-builds"
[jovian] Error: Failed to read pip environment using command: "pip --disable-pip-version-check freeze"


[jovian] Committed successfully! https://jovian.ai/karynaur/lyrics


'https://jovian.ai/karynaur/lyrics'

In [35]:
!wget https://raw.github.com/adigoryl/Styled-Lyrics-Generator-GPT2/blob/master/datasets/genius_lyrics_v2.csv

--2021-03-06 21:41:21--  https://raw.github.com/adigoryl/Styled-Lyrics-Generator-GPT2/blob/master/datasets/genius_lyrics_v2.csv
Resolving raw.github.com (raw.github.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.github.com (raw.github.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/adigoryl/Styled-Lyrics-Generator-GPT2/blob/master/datasets/genius_lyrics_v2.csv [following]
--2021-03-06 21:41:22--  https://raw.githubusercontent.com/adigoryl/Styled-Lyrics-Generator-GPT2/blob/master/datasets/genius_lyrics_v2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-03-06 21:41:23 ERROR 404: Not Found.

